In [ ]:
%pylab inline
from parcels import FieldSet, Field, ParticleSet, JITParticle, AdvectionRK4, ErrorCode, Variable
import cartopy
from glob import glob
import matplotlib.pyplot as plt
from matplotlib import colors
import numpy as np
import xarray as xr
from netCDF4 import Dataset
import math as math
import sys
sys.path.insert(1, '/home/sypmauu/GalapagosProject/scripts/functions')
from ParticlePlotFunctions import *
%matplotlib inline

In [ ]:
# compute 2008 year mean of MITgcm data

Ufiles = '/home/sypmauu/GalapagosProject/data/MIT4km/RGEMS3_2008_Surf.nc'
Xfiles = '/home/sypmauu/GalapagosProject/data/MIT4km/RGEMS3_Surf_grid.nc'
dfile = Dataset(Xfiles)
lon_MIT = dfile.variables['XG'][:]
lat_MIT = dfile.variables['YG'][:]

ddates = range(0,365)

Umean = np.zeros([len(lat_MIT),len(lon_MIT)], dtype=float)
Vmean = np.zeros([len(lat_MIT),len(lon_MIT)], dtype=float)
Tmean = np.zeros([len(lat_MIT),len(lon_MIT)], dtype=float)

teller = 0
dfile = Dataset(Ufiles)
for t in ddates:
    teller += 1
    print(teller)
    Umean += dfile.variables['UVEL'][t,:,:]
    Vmean += dfile.variables['VVEL'][t,:,:]
    Tmean += dfile.variables['THETA'][t,:,:]

Umean_MIT = Umean/teller
Vmean_MIT = Vmean/teller
Tmean_MIT = Tmean/teller

Umean_MIT[Umean_MIT==0]=nan
Vmean_MIT[Vmean_MIT==0]=nan
Tmean_MIT[Tmean_MIT==0]=nan


In [ ]:
# compute 2008 year mean of CMEMS data

ddir = '/projects/0/topios/hydrodynamic_data/CMEMS/GLOBAL_REANALYSIS_PHY_001_030/'
datafiles = sorted(glob(ddir+'mercatorglorys12v1_gl12_mean_2008*.nc'))

latdim = np.arange(800,1100)
londim = np.arange(700,1500)
dfile = Dataset(datafiles[0])
lon_CMEMS = dfile.variables['longitude'][londim]
lat_CMEMS = dfile.variables['latitude'][latdim]

ddates = range(0,365)

Umean = np.zeros([len(lat_CMEMS),len(lon_CMEMS)], dtype=float)
Vmean = np.zeros([len(lat_CMEMS),len(lon_CMEMS)], dtype=float)
Tmean = np.zeros([len(lat_CMEMS),len(lon_CMEMS)], dtype=float)

teller = 0
for t in ddates:
    teller += 1
    print(teller)
    dfile = Dataset(datafiles[t])
    Umean += dfile.variables['uo'][0,0,latdim,londim]
    Vmean += dfile.variables['vo'][0,0,latdim,londim]
    Tmean += dfile.variables['thetao'][0,0,latdim,londim]

Umean_CMEMS = Umean/teller
Vmean_CMEMS = Vmean/teller
Tmean_CMEMS = Tmean/teller

Umean_CMEMS[Umean_CMEMS<=-100]=nan
Vmean_CMEMS[Vmean_CMEMS<=-100]=nan
Tmean_CMEMS[Tmean_CMEMS<=-100]=nan

In [ ]:
# compute 2008 year mean of NEMO

ddir = '/projects/0/topios/hydrodynamic_data/NEMO-MEDUSA/ORCA0083-N006/means/'
ddates = range(584,657) #year 2008
nz = 0
map_crop = [2000, 3001, 1000, 2001]

Field = ReadNemoMean(ddir, ddates, nz, map_crop)

lon_NEMO = Field['lon']
lat_NEMO = Field['lat']
Umean_NEMO = Field['Umean']
Vmean_NEMO = Field['Vmean']
Tmean_NEMO = Field['Tmean']

In [ ]:
# save data

np.savez('meanfields2008_MIT.npz',
         lon_MIT = lon_MIT,
         lat_MIT = lat_MIT,
         Umean_MIT = Umean_MIT,
         Vmean_MIT = Vmean_MIT,
         Tmean_MIT = Tmean_MIT)

np.savez('meanfields2008_NEMO.npz',
         lon_NEMO = lon_NEMO,
         lat_NEMO = lat_NEMO,
         Umean_NEMO = Umean_NEMO,
         Vmean_NEMO = Vmean_NEMO,
         Tmean_NEMO = Tmean_NEMO)

np.savez('meanfields2008_CMEMS.npz',
         lon_CMEMS = lon_CMEMS,
         lat_CMEMS = lat_CMEMS,
         Umean_CMEMS = Umean_CMEMS,
         Vmean_CMEMS = Vmean_CMEMS,
         Tmean_CMEMS = Tmean_CMEMS)